# **Investment Portfolio Management**

## **About the scenario**
This scenario demonstrates best practices in a data engineering context, where notebooks are used to orchestrate complex tasks involving real-time data fetching, computation, and report delivery. The notebook is modular, and secure, employing batch processing, error handling, and proper separation of concerns.

In this scenario, we will:

1. Ingest a CSV file containing the user’s investment portfolio.
2. Fetch real-time stock prices via an API using *Function Calling*.
3. Perform calculations on the portfolio, including total value and percentage changes, using *Code Interpreter*.
4. Generate a detailed report summarizing the portfolio's performance.
5. Save the report to Azure Blob Storage using *Function Calling*.

## **Time**
You should expect to spend 10-15 minutes building and running this scenario. 

## **Before you begin**

#### Step 1: Install required libraries

In [49]:
# Install the packages
%pip install -r ./requirements.txt

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


## **Azure OpenAI setup**

#### Step 1: Setting Up the Environment

Before we begin, we'll load necessary environment variables from a `.env` file. These variables contain sensitive information such as API keys and endpoint URLs. Ensure your `.env` file is properly configured in the `.env/.development.env` format. Here we will also import key libraries needed to support this notebook.


In [50]:
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv(dotenv_path=".env/.development.env")

# Retrieve the secrets
__AZURE_OPENAI_API_KEY = os.getenv("AZURE_OPENAI_API_KEY")
__AZURE_OPENAI_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")
__AZURE_OPENAI_DEPLOYMENT = os.getenv("AZURE_OPENAI_DEPLOYMENT")
__AZURE_OPENAI_API_VERSION = os.getenv("AZURE_OPENAI_API_VERSION")

# Verify environment variables
if not all([__AZURE_OPENAI_API_KEY, __AZURE_OPENAI_ENDPOINT, __AZURE_OPENAI_DEPLOYMENT, __AZURE_OPENAI_API_VERSION]):
    raise EnvironmentError("One or more environment variables are missing. Please check the .env file.")

print("Environment variables loaded successfully.")

Environment variables loaded successfully.


#### Step 2: Initializing the Azure OpenAI Client

Next, we’ll initialize the Azure OpenAI client. This client allows us to interact with Azure OpenAI's file management APIs. You’ll need your API key, endpoint, and API version, which we just loaded from the environment.

In [51]:
from openai import AzureOpenAI, OpenAIError

# Initialize the AzureOpenAI client
try:
    openai_client = AzureOpenAI(
        api_key=__AZURE_OPENAI_API_KEY,
        api_version=__AZURE_OPENAI_API_VERSION,
        azure_endpoint=__AZURE_OPENAI_ENDPOINT
    )
    print("AzureOpenAI client initialized.")
except OpenAIError as e:
    raise ConnectionError(f"Failed to initialize AzureOpenAI client: {e}")


AzureOpenAI client initialized.


#### Step 3 - Upload Support File(s)

In this section, we’ll define a function to check if a file already exists in Azure OpenAI. If it exists, we’ll delete it before uploading the new version. This function will handle the complete upload process.


In [52]:
# Directory containing files to upload
directory = "data"
purpose = "assistants"

# Check if the directory exists
if not os.path.isdir(directory):
    print(f"Directory '{directory}' does not exist.")
else:
    # Process each file in the directory
    for file_name in os.listdir(directory):
        file_path = os.path.join(directory, file_name)

        # Skip if not a file
        if not os.path.isfile(file_path):
            print(f"Skipping non-file item: {file_name}")
            continue

        try:
            # Check if the file already exists on OpenAI
            existing_files = openai_client.files.list()
            file_exists = False
            for f in existing_files:
                if f.filename == file_name and f.purpose == purpose:
                    # Delete the existing file
                    openai_client.files.delete(file_id=f.id)
                    print(f"Deleted existing file: {file_name}")
                    file_exists = True
                    break

            # Upload the new file
            with open(file_path, "rb") as file_data:
                portfolio_file = openai_client.files.create(
                    file=file_data,
                    purpose=purpose
                )
            print(f"Uploaded file: {file_name}")
        
        except OpenAIError as e:
            print(f"Error processing file '{file_name}': {e}")
        except Exception as e:
            print(f"Unexpected error with file '{file_name}': {e}")

Deleted existing file: investment_portfolio.csv
Uploaded file: investment_portfolio.csv


## **Function Calling**

#### Step 1: Define function for Assistant
The Get Latest Stock Prices function retrieves the stock data for a specified `ticker symbol` using the yfinance library, specifically pulling the latest data for the last trading day.

The `fetch_stock_price` function uses the `yfinance` library to retrieve stock information, including:
- Company name
- Latest open price
- Latest close price

In [53]:
import yfinance as yf

def fetch_stock_price(ticker_symbol):
    """
    Fetch the latest stock price and opening price for a given ticker symbol.

    Parameters:
    - ticker_symbol (str): The ticker symbol of the stock to retrieve data for.

    Returns:
    - dict: A dictionary containing the following keys:
        - "company_name" (str): The name of the company corresponding to the ticker symbol.
        - "ticker" (str): The ticker symbol provided as input.
        - "open_price" (float): The opening price of the stock for the latest trading day.
        - "latest_close_price" (float): The closing price of the stock for the latest trading day.
        - "error" (str): An error message if an issue occurred, or if no data was found.
        
    Example:
    >>> fetch_stock_price("AAPL")
    {'ticker': 'AAPL', 'open_price': 145.3, 'latest_close_price': 148.9}
    """
    
    # Define the default return structure
    default_response = {
        "company_name": None,
        "ticker": ticker_symbol,
        "latest_open_price": None,
        "latest_close_price": None,
        "error": None
    }

    try:
        # Fetch the stock's trading history for the last day
        stock = yf.Ticker(ticker_symbol)
        stock_data = stock.history(period="1d")

        # Check if the data is empty, indicating an invalid ticker or no data available
        if stock_data.empty:
            return {"error": f"No data found for ticker symbol: {ticker_symbol}"}

         # Extract the company name from the 'info' dictionary
        company_name = stock.info.get('longName', "Company name not available")

        # Extract the latest available open and close prices
        latest_open_price = stock_data['Open'].iloc[-1]
        latest_close_price = stock_data['Close'].iloc[-1]
        

        return str(round(latest_close_price,3))
        #return f"{company_name} ({ticker_symbol}) stock opened at {round(latest_open_price,3)} and closed at {round(latest_close_price)}.\n"

        #return {
        #    "company_name": company_name,
        #    "ticker": ticker_symbol,
        #    "latest_open_price": latest_open_price,
        #    "latest_close_price": latest_close_price,
        #    "error": None
        #}

    except KeyError as e:
        return {**default_response, "error": f"Data missing for key: {e}. Verify the ticker symbol."}

    except Exception as e:
        return {**default_response, "error": f"Unexpected error: {type(e).__name__}: {e}"}
    
print("Function defined successfully.")

Function defined successfully.


#### Step 2: Integrating defined function into the Assistant
Create the `fetch_stock_price` function configuration to be used within the code interpreter. By adding this tool to the Assistant, we enable it to retrieve and interpret stock data when a user prompts it with related queries.

In [54]:
tools_list = [
    {"type": "code_interpreter"},
    {
        "type": "function",
        "function": {
            "name": "fetch_stock_price",
            "description": "Retrieve the latest closing price of a stock using its ticker symbol.",
            "parameters": {
                "type": "object",
                "properties": {"ticker_symbol": {"type": "string", "description": "The ticker symbol of the stock"}},
                "required": ["ticker_symbol"],
            },
        },
    }
]

print("Tools list defined successfully.")

Tools list defined successfully.


## **Assistant**

#### Step 1: Creating the Investment Management Assistant
In this step, we create an assistant with specific tools, including a code interpreter, to answer investment-related questions. This assistant will help analyze and provide insights based on the uploaded portfolio file.


In [55]:
# Create the assistant with code interpreter enabled
try:
    assistant = openai_client.beta.assistants.create(
        name="Investment Management Assistant",
        instructions=(
            "You are an expert investment analyst. "
            "Use your knowledge base to answer questions about personal investment portfolio management."
        ),
        model=__AZURE_OPENAI_DEPLOYMENT,
        tools=tools_list
    )
    print("Assistant created successfully:", assistant)
except OpenAIError as e:
    print("Error creating assistant:", e)

Assistant created successfully: Assistant(id='asst_0F1BeDTYLExyx8GfFpuJCrsT', created_at=1731442454, description=None, instructions='You are an expert investment analyst. Use your knowledge base to answer questions about personal investment portfolio management.', metadata={}, model='gpt-4o', name='Investment Management Assistant', object='assistant', tools=[CodeInterpreterTool(type='code_interpreter'), FunctionTool(function=FunctionDefinition(name='fetch_stock_price', description='Retrieve the latest closing price of a stock using its ticker symbol.', parameters={'type': 'object', 'properties': {'ticker_symbol': {'type': 'string', 'description': 'The ticker symbol of the stock'}}, 'required': ['ticker_symbol']}, strict=False), type='function')], response_format='auto', temperature=1.0, tool_resources=ToolResources(code_interpreter=ToolResourcesCodeInterpreter(file_ids=[]), file_search=None), top_p=1.0)


#### Step 2: Starting a New Conversation Thread
Let's create a new thread to handle user interactions. Each thread allows for a dedicated conversation with the assistant.

In [56]:
# Create a conversation thread
try:
    thread = openai_client.beta.threads.create()
    print("Thread created:", thread)
except OpenAIError as e:
    print("Error creating thread:", e)

Thread created: Thread(id='thread_nNEa3nenX9EG7xdtnc8md9xA', created_at=1731442454, metadata={}, object='thread', tool_resources=ToolResources(code_interpreter=None, file_search=None))


#### Step 3: Adding User Message
In this step, we add a question to the thread. For this demonstration, we're asking which stock ticker has the most investment in the portfolio.

In [57]:
# Define the user question
#prompt_content = "What ticker symbol has the most stock? Which has the most investment?"

#prompt_content = "What is the latest closing price of AAPL?"

prompt_content = "What is the latest closing price for MSFT? What is my total investment for MSFT as of today?"

# Add the question to the thread
try:
    message = openai_client.beta.threads.messages.create(
        thread_id=thread.id,
        role="user",
        content=prompt_content,
        attachments=[  # Add files by using the attachments parameter
            {"file_id": portfolio_file.id, "tools": [{"type": "code_interpreter"}]}
        ],
    )
    print("User question added:", message)
except OpenAIError as e:
    print("Error adding user question:", e)


User question added: Message(id='msg_jdyTyCObzhD0NJogt2KI9rH8', assistant_id=None, attachments=[Attachment(file_id='assistant-byvyRpwiVS4ThTl3blekxTvT', tools=[CodeInterpreterTool(type='code_interpreter')])], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='What is the latest closing price for MSFT? What is my total investment for MSFT as of today?'), type='text')], created_at=1731442454, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='user', run_id=None, status=None, thread_id='thread_nNEa3nenX9EG7xdtnc8md9xA')


#### Step 4: Running the Assistant
Now that the assistant and thread are set up, we'll initiate the assistant's response process. This will analyze the user prompt and provide insights based on the investment portfolio data.


In [58]:
# Initiate the assistant's response
try:
    run = openai_client.beta.threads.runs.create(
        thread_id=thread.id,
        assistant_id=assistant.id,
        instructions=prompt_content,
    )
    print("Run started:", run)
except OpenAIError as e:
    print("Error starting run:", e)

Run started: Run(id='run_nTlD90Zrnpzi5vr3x2l59RpV', assistant_id='asst_0F1BeDTYLExyx8GfFpuJCrsT', cancelled_at=None, completed_at=None, created_at=1731442454, expires_at=1731443054, failed_at=None, incomplete_details=None, instructions='What is the latest closing price for MSFT? What is my total investment for MSFT as of today?', last_error=None, max_completion_tokens=None, max_prompt_tokens=None, metadata={}, model='gpt-4o', object='thread.run', required_action=None, response_format='auto', started_at=None, status='queued', thread_id='thread_nNEa3nenX9EG7xdtnc8md9xA', tool_choice='auto', tools=[CodeInterpreterTool(type='code_interpreter'), FunctionTool(function=FunctionDefinition(name='fetch_stock_price', description='Retrieve the latest closing price of a stock using its ticker symbol.', parameters={'type': 'object', 'properties': {'ticker_symbol': {'type': 'string', 'description': 'The ticker symbol of the stock'}}, 'required': ['ticker_symbol']}, strict=False), type='function')], t

#### Step 5: Checking for Results
The assistant may take some time to analyze and respond. We'll use a polling loop to periodically check the assistant's status and retrieve the answer once it's available.

**>>>>>>> TO DO: create a function to simply sending multiple prompts**

In [59]:
import json
import time


available_functions = {"fetch_stock_price": fetch_stock_price}

while True:

    # Retrieve the run status
    run = openai_client.beta.threads.runs.retrieve(
        thread_id=thread.id,
        run_id=run.id
    )
    print(run.model_dump_json(indent=4))

    if run.status == "failed":
         print("Assistant run failed. Please try again.")
         break
    elif run.status == "completed":
        messages = openai_client.beta.threads.messages.list(thread_id=thread.id)

        # Loop through messages and print content based on role
        for msg in messages.data:
            role = msg.role
            content = msg.content[0].text.value
            print(f"{role.capitalize()}: {content}")
        break
    elif run.status == "requires_action":
        print("Function Calling ...")
        tool_responses = []
        if (
            run.required_action.type == "submit_tool_outputs"
            and run.required_action.submit_tool_outputs.tool_calls is not None
        ):
            tool_calls = run.required_action.submit_tool_outputs.tool_calls

            for call in tool_calls:
                if call.type == "function":
                    if call.function.name not in available_functions:
                        raise Exception("Function requested by the model does not exist")
                    function_to_call = available_functions[call.function.name]
                    tool_response = function_to_call(**json.loads(call.function.arguments))
                    tool_responses.append({"tool_call_id": call.id, "output": tool_response})
                    print(f"Function '{call.function.name}' called successfully. \nOutput: {tool_response}\n")

        run = openai_client.beta.threads.runs.submit_tool_outputs(
            thread_id=thread.id, run_id=run.id, tool_outputs=tool_responses
        )

        print("Function called successfully.")
    else:
            print("Waiting for the Assistant to process...")
            time.sleep(5)

{
    "id": "run_nTlD90Zrnpzi5vr3x2l59RpV",
    "assistant_id": "asst_0F1BeDTYLExyx8GfFpuJCrsT",
    "cancelled_at": null,
    "completed_at": null,
    "created_at": 1731442454,
    "expires_at": 1731443054,
    "failed_at": null,
    "incomplete_details": null,
    "instructions": "What is the latest closing price for MSFT? What is my total investment for MSFT as of today?",
    "last_error": null,
    "max_completion_tokens": null,
    "max_prompt_tokens": null,
    "metadata": {},
    "model": "gpt-4o",
    "object": "thread.run",
    "required_action": null,
    "response_format": "auto",
    "started_at": null,
    "status": "queued",
    "thread_id": "thread_nNEa3nenX9EG7xdtnc8md9xA",
    "tool_choice": "auto",
    "tools": [
        {
            "type": "code_interpreter"
        },
        {
            "function": {
                "name": "fetch_stock_price",
                "description": "Retrieve the latest closing price of a stock using its ticker symbol.",
            

## **Cleanup Resources**
To avoid creating redundant resources and ensure a clean environment, this cell deletes the assistant, thread, and any other created resources. Run this cell at the end of your session to clean up.


In [60]:
response = openai_client.beta.assistants.delete(assistant.id)
print(response)

# Optionally delete any other temporary files or datas
# Note: Any uploaded files to OpenAI could also be cleaned up if needed


AssistantDeleted(id='asst_0F1BeDTYLExyx8GfFpuJCrsT', deleted=True, object='assistant.deleted')


----------------

## Scratch Pad

In [61]:
"""
import time
from IPython.display import clear_output

start_time = time.time()

status = run.status

while status not in ["completed", "cancelled", "expired", "failed"]:
    time.sleep(5)
    run = openai_client.beta.threads.runs.retrieve(thread_id=thread.id,run_id=run.id)
    print("Elapsed time: {} minutes {} seconds".format(int((time.time() - start_time) // 60), int((time.time() - start_time) % 60)))
    status = run.status
    print(f'Status: {status}')
    clear_output(wait=True)

messages = openai_client.beta.threads.messages.list(
  thread_id=thread.id
) 

print(f'Status: {status}')
print("Elapsed time: {} minutes {} seconds".format(int((time.time() - start_time) // 60), int((time.time() - start_time) % 60)))
print(messages.model_dump_json(indent=2))
"""

'\nimport time\nfrom IPython.display import clear_output\n\nstart_time = time.time()\n\nstatus = run.status\n\nwhile status not in ["completed", "cancelled", "expired", "failed"]:\n    time.sleep(5)\n    run = openai_client.beta.threads.runs.retrieve(thread_id=thread.id,run_id=run.id)\n    print("Elapsed time: {} minutes {} seconds".format(int((time.time() - start_time) // 60), int((time.time() - start_time) % 60)))\n    status = run.status\n    print(f\'Status: {status}\')\n    clear_output(wait=True)\n\nmessages = openai_client.beta.threads.messages.list(\n  thread_id=thread.id\n) \n\nprint(f\'Status: {status}\')\nprint("Elapsed time: {} minutes {} seconds".format(int((time.time() - start_time) // 60), int((time.time() - start_time) % 60)))\nprint(messages.model_dump_json(indent=2))\n'

Everything after deployment --- everything else should be in notebook (data plane - private endpoint) --- before should not, its control plane (public only)

1. Portal Creation for Deployment (note iteration two, bicep code??)
2. Assistant Creation (API)
    - code to do it not portal (delete for cleanup)
3. Create Thread, Message, Thread run (instructs assistant to create messages) - wait for complete (look at SDK to do this), Listing Messages, Check history of the thread (will see my message, and one from assistant) first message response will be answer (POST/THREADS, POST/THREADS Messages, POST Threads thr)
    - POST /threads
    - POST /threads/:thread_id/messages
    - POST /threads/:thread_id/runs
    - GET /threads/:thread_id/runs/:run_id
    - GET /threads/:thread_id/messages
4. 